In [16]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [17]:
import os
os.environ['HF_HOME'] = '/root/autodl-tmp'

In [2]:
import pandas as pd

file_path = 'entities.tsv'
df = pd.read_csv(file_path, sep='\t')

/tmp/ipykernel_1883/3821747307.py:4: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')


In [11]:
df.loc[:,'name']

0                    obsolete sudden onset of severe chills
1                                         dry hacking cough
2                                   pulmonary consolidation
3                                                  enanthem
4                                                   anxiety
                                ...                        
814974                               IL-9 signaling pathway
814975            Complement activation - classical pathway
814976                                    Heme biosynthesis
814977    FAS pathway and stress induction of HSP regula...
814978          Hepatocyte growth factor receptor signaling
Name: name, Length: 814979, dtype: object

In [12]:
df.loc[:,'id']

0              SYMP:0000149
1              SYMP:0000259
2              SYMP:0000729
3              SYMP:0000746
4              SYMP:0000412
                ...        
814974     WikiPathways:WP8
814975    WikiPathways:WP81
814976    WikiPathways:WP86
814977    WikiPathways:WP89
814978    WikiPathways:WP94
Name: id, Length: 814979, dtype: object

In [19]:
import transformers_embedder as tre

tokenizer = tre.Tokenizer("microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext")

model = tre.TransformersEmbedder(
    "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext", subword_pooling_strategy="sparse", layer_pooling_strategy="mean"
)

example = "This is a sample sentence"
inputs = tokenizer(example, return_tensors=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
import torch

outputs = model(**inputs)
torch.norm(outputs.word_embeddings[:,1])

tensor(18.2025, grad_fn=<LinalgVectorNormBackward0>)